# How To Difference Two Images

_Contact: Leanne Guy, Colin Slater, Phil Marshall_

_Last edited: 2018-05-30_

_Runs with Stack release: Weekly 2018-21_

In this notebook we run the `ImageDifferenceTask` on two visit images, and inspect the result.

## Setups

In [1]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import IFrame, display, Markdown
import os

## Finding Two Images

We'll use two single sensor images from the HSC WIDE dataset, taken close together in time and at the same point on the sky. 

Data is in `/datasets/hsc/repo/rerun/DM-10404/WIDE/`, cheating can be achieved via the sqlite3 database in `/datasets/hsc/repo/registry.sqlite3`: 
```
    select * from raw limit 10
```

In [2]:
data_dir = '/datasets/hsc/repo/rerun/DM-10404/WIDE/'

In [3]:
# Create a butler from this data directory
from lsst.daf.persistence import Butler
butler = Butler(data_dir)
#print(butler.getUri)
# get all the visits over the SSP_WIDE survey
visitPointings = butler.queryMetadata('calexp',('visit', 'pointing'),{'field':'SSP_WIDE'})
#print(visitPointings)

In [4]:
# Lets try to pull up 2 images from different fields of the same pointing
# 374, 376, pointing :  814
dataId1 = {'visit': 374, 'ccd':42}
dataId2 = {'visit': 376, 'ccd':42}

In [5]:
calexp1 = butler.get('calexp', **dataId1)
calexp2 = butler.get('calexp', **dataId2)

In [6]:
# Create a display in a new browser tab:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

In [7]:
print('Open a browser window to {}/firefly/slate.html?__wsch={}'.format(server, my_channel))

Open a browser window to https://lsst-lspdev.ncsa.illinois.edu/firefly/slate.html?__wsch=womullan_test_channel


In [8]:
# Uncomment the lines below, to get a display inside this notebook:

#ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
#IFrame(ff,800,600)

In [10]:
afw_display.setDefaultBackend('firefly')
afw_display_1 = afw_display.getDisplay(frame=1,
                                    name=my_channel)
afw_display_2 = afw_display.getDisplay(frame=2,
                                    name=my_channel)

In [11]:
afw_display_1.mtv(calexp1, "Calexp %d" % dataId1['visit'])
afw_display_2.mtv(calexp2, "Calexp %d" % dataId2['visit'])

In [12]:
# C1 date
c1VisitInfo = calexp1.getInfo().getVisitInfo()
c1Date = c1VisitInfo.getDate()

# C2
c2VisitInfo = calexp2.getInfo().getVisitInfo()
c2Date = c2VisitInfo.getDate()

# Time difference 
timediff = c2Date.toPython() -c1Date.toPython()
timediff.total_seconds()

# Try to identify visits with the same exposure time 
for visit,pointing in visitPointings[:10] : 
    dataId = {'visit': visit, 'ccd':42}
    calexp = butler.get('calexp', **dataId)
    visitInfo = calexp.getInfo().getVisitInfo()
    print(visit, ",", visitInfo.getExposureTime())


374 , 30.0
376 , 200.0
378 , 30.0
380 , 200.0
382 , 30.0
384 , 200.0
386 , 30.0
388 , 200.0
390 , 30.0
392 , 200.0


Looks like `374` and `376` will do just fine - and `376`, with its longer exposure time, will make a plausible model template image.

In [13]:
image_dataId = {'visit': 374, 'ccd':42}
template_dataId = {'visit': 376, 'ccd':42}

## Image Differencing

The relevant pipeline task is [`ImageDifferenceTask`](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/imageDifference.py#L214), wrapped in the following command line task.

In [14]:
! imageDifference.py

usage: imageDifference.py input [options]

positional arguments:
  input                 path to input data repository, relative to
                        $PIPE_INPUT_ROOT

optional arguments:
  -h, --help            show this help message and exit
  --calib RAWCALIB      path to input calibration repository, relative to
                        $PIPE_CALIB_ROOT
  --output RAWOUTPUT    path to output data repository (need not exist),
                        relative to $PIPE_OUTPUT_ROOT
  --rerun [INPUT:]OUTPUT
                        rerun name: sets OUTPUT to ROOT/rerun/OUTPUT;
                        optionally sets ROOT to ROOT/rerun/INPUT
  -c [NAME=VALUE [NAME=VALUE ...]], --config [NAME=VALUE [NAME=VALUE ...]]
                        config override(s), e.g. -c foo=newfoo bar.baz=3
  -C [CONFIGFILE [CONFIGFILE ...]], --configfile [CONFIGFILE [CONFIGFILE ...]]
                        config override file(s)
  -L [LEVEL|COMPONENT=LEVEL [LEVEL|COMPONENT=LEVEL ...]], --loglevel [LEV

The command line task does not complete, in our simple visit-visit example. Instead, let's call the python commands directly. (The command line that ought to run is given below, commented out, for completeness.)

In [15]:
# ! imageDifference.py $data_dir --output 'diffim-output' --id visit=image_dataId ccd=42 --templateId visit=template_dataId --configfile diffim-config.py

In [16]:
# Set up an ImageDifferenceTask object:
from lsst.pipe.tasks.imageDifference import ImageDifferenceTask, ImageDifferenceConfig 
from lsst.ip.diffim.getTemplate import GetCalexpAsTemplateTask

config = ImageDifferenceConfig()
config.getTemplate.retarget(GetCalexpAsTemplateTask)
config.doWriteSources=False
config.doWriteSubtractedExp=False
imageDifference = ImageDifferenceTask(butler=butler, config=config)

# Prepare the science image, and the template to difference it with:
image = butler.dataRef('calexp', dataId=image_dataId)
# template = butler.dataRef('calexp', dataId=template_dataId) 
# NB: the template is not a Butler dataref objects, but instead a data ID:
template = template_dataId

# Take the difference:
diffedImage = imageDifference.run(image, templateIdList=[template])

/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/ip_diffim/15.0-8-g67a62d3/python/lsst/ip/diffim/dipoleFitTask.py:303: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pars = np.linalg.lstsq(M, B)[0]
/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/ip_diffim/15.0-8-g67a62d3/python/lsst/ip/diffim/dipoleFitTask.py:303: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pars = np.linalg.lstsq(M, B)[0]
/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/ip_diffim/15.0-8-g67a62d3/python/

In [17]:
# To help debug the above commands, try using the python debugger:
# %pdb
# (This is a toggle.)

Let's just print the resulting structure, to see what we have: 

In [18]:
diffedImage

Struct(subtractedExposure=<lsst.afw.image.exposure.exposure.ExposureF object at 0x7f39f1a53fb8>; subtractRes=Struct(matchedImage=<lsst.afw.image.maskedImage.maskedImage.MaskedImageF object at 0x7f39f1a59148>; psfMatchingKernel=<lsst.afw.math._kernel.LinearCombinationKernel object at 0x7f39f1a59180>; backgroundModel=<lsst.afw.math._functionLibrary.Chebyshev1Function2D object at 0x7f39f1a591b8>; kernelCellSet=<lsst.afw.math._spatialCell.SpatialCellSet object at 0x7f39f1a540d8>; warpedExposure=<lsst.afw.image.exposure.exposure.ExposureF object at 0x7f39f1a54110>; matchedExposure=<lsst.afw.image.exposure.exposure.ExposureF object at 0x7f39f1a4dbc8>; subtractedExposure=<lsst.afw.image.exposure.exposure.ExposureF object at 0x7f39f1a53fb8>); sources=<class 'lsst.afw.table.source.source.SourceCatalog'>
       id            coord_ra      ... ip_diffim_DipoleFit_flag_edge
                       rad         ...                              
--------------- ------------------ ... -----------------

In [19]:
# Plot the differenced image in the firefly window.
afw_display_2.mtv(diffedImage.subtractedExposure)

The layers can be adjusted in the firefly display, to see the DIAsources.